In [1]:
# price_analysis.ipynb

import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
from datetime import datetime

def fetch_price_feeds():
    """Fetch price feeds from our Django API"""
    response = requests.get('http://localhost:8000/api/price-feeds/')
    return response.json()

def create_price_dataframe():
    """Convert price feed data to pandas DataFrame"""
    data = fetch_price_feeds()
    
    # Extract feeds into a DataFrame
    df = pd.DataFrame(data['feeds'])
    
    # Convert timestamp to datetime
    df['timestamp'] = pd.to_datetime(data['timestamp'], unit='s')
    
    # Convert value to numeric
    df['value'] = pd.to_numeric(df['value'])
    
    return df

# Example analyses

def basic_analysis():
    df = create_price_dataframe()
    
    # Basic statistics
    stats = df.groupby('status')['value'].agg(['count', 'mean', 'std', 'min', 'max'])
    
    # Filter for stablecoins
    stablecoins = df[df['pair'].str.contains('USD[CT]/USD')]
    stablecoin_stats = stablecoins['value'].describe()
    
    # Find highest and lowest value assets
    top_values = df.nlargest(5, 'value')[['pair', 'value', 'base_asset']]
    
    return {
        'basic_stats': stats,
        'stablecoin_stats': stablecoin_stats,
        'top_values': top_values
    }

def analyze_price_patterns():
    df = create_price_dataframe()
    
    # Calculate price ranges and volatility metrics
    analysis = pd.DataFrame({
        'pair': df['pair'],
        'current_price': df['value'],
        'status': df['status']
    })
    
    return analysis

# Example usage:
# if __name__ == "__main__":
#     # Basic stats
#     stats = basic_analysis()
#     print("\nBasic Statistics by Status:")
#     print(stats['basic_stats'])
    
#     print("\nTop 5 Assets by Value:")
#     print(stats['top_values'])
    
#     # Price patterns
#     patterns = analyze_price_patterns()
#     print("\nPrice Analysis:")
#     print(patterns.head())

# Function to save historical data
def save_historical_data(df, filename='historical_prices.csv'):
    """
    Append current price data to historical CSV file
    """
    try:
        # Try to load existing data
        historical = pd.read_csv(filename)
        # Append new data
        historical = pd.concat([historical, df])
    except FileNotFoundError:
        # If file doesn't exist, create new
        historical = df
    
    # Save to CSV
    historical.to_csv(filename, index=False)

In [5]:
fetch_price_feeds()

{'status': 'success',
 'timestamp': 1739011584,
 'feeds': [{'pair': 'FLR/USD',
   'base_asset': 'Flare',
   'status': '🟢',
   'feed_id': '0x01464c522f55534400000000000000000000000000',
   'value': '0.0211984',
   'decimals': 7},
  {'pair': 'SGB/USD',
   'base_asset': 'Songbird',
   'status': '🟡',
   'feed_id': '0x015347422f55534400000000000000000000000000',
   'value': '0.00617988',
   'decimals': 8},
  {'pair': 'BTC/USD',
   'base_asset': 'Bitcoin',
   'status': '🟢',
   'feed_id': '0x014254432f55534400000000000000000000000000',
   'value': '96178.06',
   'decimals': 2},
  {'pair': 'XRP/USD',
   'base_asset': 'XRP',
   'status': '🟢',
   'feed_id': '0x015852502f55534400000000000000000000000000',
   'value': '2.412131',
   'decimals': 6},
  {'pair': 'LTC/USD',
   'base_asset': 'Litecoin',
   'status': '🟢',
   'feed_id': '0x014c54432f55534400000000000000000000000000',
   'value': '102.83378',
   'decimals': 5},
  {'pair': 'XLM/USD',
   'base_asset': 'Stellar',
   'status': '🟡',
   'feed_i